<a href="https://colab.research.google.com/github/karlssoj/compvis/blob/Exempel3_Bildklassificering/X_Y_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1> Klassificeringsexempel med faltningsnätverk (CNN) implementerat med Keras och Tensorflow </h1>

In [2]:
!git clone -b Exempel3_Bildklassificering https://github.com/karlssoj/compvis.git

fatal: destination path 'compvis' already exists and is not an empty directory.


I det här simplata exemplet tränar vi in en modell att kunna se skillnad mellan X och O. I katalogen X_O_training finns 2 kataloger O och X. Vardera katalog innehåller 4 träningsbilder på X respektive O. Sedan har vi en bild i samma katalog som denna notebookfil,"test_image.jpg" som vi använder för att testa om modellen gissar rätt. Observera att för att den här modellen ska fungera krävs det bilder där X och Y fyller hela bakrunden. För att modellen ska fungera med X och Y av olika storlekar placerade på olika ställen i bakgrunden krävs en bättre dataset.

Skriptet är testkört i google colab (men kan nog även köras lokalt). Om du kör på colab.

<b> 1. Importerar OpenCV och Matplotlibbelipippeli

In [ ]:
import matplotlib.pyplot as plt
import cv2 as cv

<b> 2. Importerar ImageDataGenerator från Keras-biblioteket för att kunna hantera träningsbilderna i X_O_training/o och X_O_training/x

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

<b> 3. Skapar ett ImageDataGenerator-objekt med vissa inställningar för hur de inlästa träningsbilderna ska "randomiseras"

In [ ]:
image_gen = ImageDataGenerator(rotation_range=30,    #slumpmässiga roteringar av bilderna i grader
                             width_shift_range=0.1,  #slumpmässig skiftning i x-led (ett värde mellan 0 och 1)
                             height_shift_range=0.1, #slumpmässig skiftning i y-led (ett värde mellan 0 och 1)
                             rescale=1/255,          #skalar ner ett 8-bitars pixelvärde (0-255) till ett värde mellan 0 och 1
                             zoom_range=0.6,         #slumpmässig zoom
                             horizontal_flip=True)   #"flippar" bilden slumpmässigt i horisontalt läge

<b> 4. Läser in alla träningsbilder på X och O från respektive kataloger i X_O_training-katalogen. Alla bilder får storleken 100x100 pixlar

In [ ]:
train_image_gen = image_gen.flow_from_directory('/content/drive/MyDrive/XYExample/X_O_training',
                                                target_size=(100,100))

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/XYExample/X_O_training'

<b> 5. Träningsbilderna har automatiskt markerats enligt namnet på katalogerna de finns sparade i, dvs. O har labeln 0 och X har labeln 1

In [ ]:
print(train_image_gen.class_indices)

<b> 6. Importerar behövliga bibliotek från Keras för att kunna skapa en CNN-modell

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

<b> 7. Skapar ett faltningslager (convolutional layer) bestående av 32 olika filters. Som aktiveringsfunktion används ReLU (för att bli av med alla negativa pixelvärden<br><br>

<b> 8. Kör MaxPooling med 2x2 fönsterstorlek och hopp (stride) = 2<br><br>

<b> 9. Formaterar om alla filtrerade bilder (i detta fall 32 stycken) till en enda array = flatten<br><br>

<b> 10. Till slut mynnar allt ut i 2 neuroner i "fully connected layer", en för X och en för Y

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(100,100,3), activation = 'relu')) #Faltningslager med 32 filter
model.add(MaxPooling2D(pool_size = (2, 2)))                                 #MaxPooling 2x2 fönsterstorlek och hopp (stride) = 2
model.add(Flatten())                                                        #Formaterar om alla filtrerade blder (32 stycken) till en array
model.add(Dense(units = 2, activation = "sigmoid"))                         #2 neuroner på slutet

<b> 11. "Kompilerar" modellen. Optimizer är den metod man vill använda för att optimera alla vikter i nätet. Adam är en variant av gradient descent. Loss definierar "loss function", dvs. vad man vill använda för metod för att beräkna feluppskattningen under inträningen.

In [ ]:
model.compile(optimizer = 'adam',
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

In [ ]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 98, 98, 32)        896       
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 49, 49, 32)        0         
 g2D)                                                            
                                                                 
 flatten_3 (Flatten)         (None, 76832)             0         
                                                                 
 dense_3 (Dense)             (None, 2)                 153666    
                                                                 
Total params: 154562 (603.76 KB)
Trainable params: 154562 (603.76 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


<b> 12. Startar själva inträningen på testdataset:en. Epochs definierar hur många ggr inlärningsalgoritmen kommer att gå igenom hela dataset:en.

In [ ]:
results = model.fit(train_image_gen, epochs=100)

In [ ]:
plt.plot(results.history['accuracy'])

<b> 13. Vi laddar in en testbild (En bild som inte finns i träningsdataset:en) för att se hur bra vår modell fungerar! Observera att i detta simpla exempel så måste X eller O täcka mer eller mindre hela bilden och bakgrunden bör (antagligen) vara vit för att det ska fungera. För att få modellen att hitta X eller O även om de är skrivna i mindre storlek och i olika bakgrunder så måste träningsdataset:en utökas.

In [ ]:
import numpy as np
import keras.utils as image
test_image = '/content/drive/MyDrive/XYExample/test_image1.jpg'
test_image = image.load_img(test_image, target_size=(100,100)) #laddar testbilden och sätter storleken till samma som träningsbilderna
test_image = image.img_to_array(test_image)                    #konverterar till array-format

test_image = np.expand_dims(test_image, axis=0)
test_image = test_image/255                                    #Ändrar pixelvärden från 8-bitar (0-255) till värden mellan 0 och 1

<b> 14. Vi startar "predikteringen" dvs. Som svar får vi konfidensvärden som berättar hur stor sannolikhten är att bilden innehåller ett X och hur stor sannolikheten är att bilden innehåller ett O

In [ ]:
prob = model.predict(test_image)

In [ ]:
print(prob)

In [ ]:
print("Sannolikheten att testbilden innehåller O: " + str(prob[0][0]))
print("Sannolikheten att testbilden innehåller X: " + str(prob[0][1]))